In [4]:
# We have the scrapped the tables from sec.report website and stored the important ones in our database

from dotenv import load_dotenv
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import pymongo,json,os
load_dotenv()
MONGODB_URI = os.environ['MONGODB_URI']
client = pymongo.MongoClient(MONGODB_URI)
db1=client['tech-meet']
db=db1['form-data']
database=db1['info']
url="https://sec.report/CIK/"
headers={'user-agent':'Mozilla/5.0 (X11; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
db=list(db.find())

In [5]:
count=0
for db2 in db:
    count+=1
    orig=db2
    furl=url+str(db2['cik'])
    htm=requests.get(furl,headers=headers)
    if(htm.status_code==200):
        re=pd.read_html(htm.text)
        cnt1=0
        cvv=True
        info={}
        soup=BeautifulSoup(htm.text,'lxml')
        match=soup.find('div',class_='panel-body').text
        db2['summary']=str(match)
        for r in re:           
            col=r.columns
            if 'Form' in col:
                continue
            else:
                
                if r.shape[1]==2:
                    cnt1+=1                  
                if cnt1==2:
                    
                    for t in range(r.shape[0]):
                        info[str(r.iloc[t,0])]=str(r.iloc[t,1])
                if cvv and r.shape[1]==4: 
                    for t in range(r.shape[0]):
                        info[str(r.iloc[t,1])]=str(r.iloc[t,0])
                    cvv=False
        db2['info']=info
        try:                      
            database.insert_one(db2)
        except:
            orig['summary']=match
            database.insert_one(orig)
            print(db2['cik'])
            continue
print(count)                    

315
